# Quantum variable cell relaxation of LaH10

In this tutorial we will replicate the results of the Nature paper [Errea et al XXX], where they show that quantum effects make the rombohedral structure R-3m of LaH10 to collapse into the higher symmetry Fm-3m. 

*Note*: We will use underconverged parameters to run the calculation on a local computer, for production runs you must study the convergence, especially with K point sampling and the supercell size.

For this tutorial, we provide a dynamical matrix, obtained by a sscha relaxation at fixed cell (see tutorial on H3S for example). In the following I set up the espresso parameter

In [1]:
%pylab
from __future__ import print_function

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import ase
from ase.calculators.espresso import Espresso

import cellconstructor as CC
import cellconstructor.Phonons

import sscha, sscha.Ensemble, sscha.SchaMinimizer, sscha.Relax

pseudo = {"H": "H.pbe-rrkjus_psl.1.0.0.UPF",
         "La" : "La.pbe-spfn-rrkjus_psl.1.0.0.UPF"}
input_params = {"tstress" : True,
                "tprnfor" : True,
                "ecutwfc" : 35,
                "ecutrho" : 350,
                "mixing_beta" : 0.2,
                "conv_thr" : 1e-9,
                "degauss" : 0.02,
                "smearing" : "mp",
                "pseudo_dir" : ".",
                "occupations" : "smearing",
               "disk_io" : "none"}

k_points = (8,8,8)
k_offset = (1,1,1)

espresso_calc = Espresso(pseudopotentials = pseudo, input_data = input_params, 
                        kpts = k_points, koffset = k_offset)

In [3]:
# We now load the dynamical matrix
dyn = CC.Phonons.Phonons("dyn")
dyn.Symmetrize() #Enforce the sum rule

# We prepare the ensemble
ensemble = sscha.Ensemble.Ensemble(dyn, T0 = 0, supercell = dyn.GetSupercell())

# We prepare the sscha minimizer
minim = sscha.SchaMinimizer.SSCHA_Minimizer(ensemble)

# We set up the minimization parameters
minim.min_step_dyn = 0.05     # The minimization step on the dynamical matrix
minim.min_step_struc = 0.05   # The minimization step on the structure
minim.kong_liu_ratio = 0.5     # The parameter that estimates whether the ensemble is still good

We setup the standard sscha minimiztion. Now we must prepare the calculator for the automatic relaxation exactly like we did for the H3S example.
Remember you can always specify a cluster for the automatic calculation:

*In the case you want to use this cluster, please remember to upload the pseudos on the cluster working directory!*

In [4]:
# Here we prepare a cluster
# Here we configure the cluster object MARCONI
import sscha.Cluster
my_hpc = sscha.Cluster.Cluster(pwd = None)

# We setup the connection info
my_hpc.hostname = "ekhi" # The command to connect via ssh to the cluster
#my_hpc.account_name = "IscrB_COMRED" # The name of the project for the computation
my_hpc.workdir = "/scratch/lorenzo" # the directory in which the calculations are performed

# Now we need to setup the espresso
# First we must tell the cluster where to find him:
my_hpc.binary = "pw.x -npool NPOOL -i  PREFIX.pwi > PREFIX.pwo"
# Then we need to specify if some modules must be loaded in the submission script
my_hpc.load_modules = """
# Here this is a bash script at the beginning of the submission
# We can load modules

module load QuantumESPRESSO
export OMP_NUM_THREADS=1
"""

# All these information are independent from the calculation
# Now we need some more specific info, like the number of processors, pools and other stuff
my_hpc.n_cpu = 32 # We will use 32 processors
my_hpc.n_nodes = 1 #In 1 node
my_hpc.n_pool = 16 # This is an espresso specific tool, the parallel CPU are divided in 4 pools

# We can also choose in how many batch of jobs we want to submit simultaneously, and how many configurations for each job
my_hpc.batch_size = 10
my_hpc.job_number = 20
# In this way we submit 40 jobs, each one with 10 configurations

# We give 25 seconds of timeout
my_hpc.set_timeout(25)

# We can specify the time limit for each job,
my_hpc.time = "00:30:00" # 30 minutes

In [5]:
relax = sscha.Relax.SSCHA(minim, ase_calculator = espresso_calc,
                         N_configs = 200,
                         max_pop = 10,
                         save_ensemble = True,
                         cluster = my_hpc)

This time we are interested in plotting the symmetry analisys, as we evolve the minimization

In [6]:
import spglib
print ("The original spacegroup is:", spglib.get_spacegroup(dyn.structure.get_ase_atoms(), 0.1))

The original spacegroup is: R-3m (166)


In [7]:
# we define a function that prints the space group during the optimization
space_groups = []
def print_spacegroup(minim):
    spgroup = spglib.get_spacegroup(minim.dyn.structure.get_ase_atoms(), 0.1)
    space_groups.append(spgroup)
    
    # We can save them in the output at each minimization step
    f = open("space_group.dat", "w")
    f.writelines(["{}) {}\n".format(i+1, x) for i,x in enumerate(space_groups)])
    f.close()
    
relax.setup_custom_functions(custom_function_post = print_spacegroup)

Now we are ready to start a variable cell relaxation. There are two different variable cell relaxations implemented in the SSCHA code: target pressure or fixed volume.
In the target pressure calculation, we will use the SSCHA to adapt the cell until the stress stress tensor is uniform and reproduces the pressure we want. In this calculation the volume will change. In the fixed volume, instead, the SSCHA will optimize the cell parameters, but it will keep the overall volume unchanged.

We will use the latter in this example, but feel free to experiment by selecting fix_volume to false, and manually change the target_pressure argument. I prepared another command (commented) to perform a variable cell relaxation with target pressure.

The static_bulk_modulus is a flag that allow the program to estimate, given the stress tensor, how to change the unit cell to reach the optimal value in the lowest number of steps. A good value is the static bulk modulus, that is the derivative pressure with respect to the volume times the volume. The code expects it in GPa.
Usually for high pressure materials, the bulk modulus is around hundreds of GPa, while for ice at ambient pressure is about 10 GPa. A high value of the bulk modulus will mean a slower change in the unit cell, so if you have the fealing that the unit cell is not changing a lot between minimization, try to reduce it.  

In [8]:
# Now we can run the calculation!!!
# In this case we fix the volume (we optimize lattice parameters)
# Bu
import os
if not os.path.exists("ensembles"):
    os.mkdir("ensembles")
relax.vc_relax(fix_volume = True, static_bulk_modulus = 500, ensemble_loc = "ensembles")
#relax.vc_relax(target_press = 120, static_bulk_modulus = 500, ensemble_loc = "ensembles")

/home/pione/anaconda2/lib/python2.7/site-packages/sscha/Ensemble.py:671: ComplexWarning: Casting complex values to real discards the imaginary part
  self.sscha_forces[i,:,:] = force
/home/pione/anaconda2/lib/python2.7/site-packages/sscha/Ensemble.py:839: ComplexWarning: Casting complex values to real discards the imaginary part
  self.sscha_energies[i], self.sscha_forces[i, :,:] = new_super_dyn.get_energy_forces(self.structures[i], real_space_fc = new_super_dyn.dynmats[0], displacement = self.u_disps[i, :])


Time elapsed to prepare the rho update: 0.146378040314 s
(of which to update sscha energies and forces: 0.0161 s)
(of which computing the Upsilon matrix: 0.0474 s)
Time elapsed to update weights the sscha energies, forces and displacements: 0.0664892196655 s
(of which to update the weights): 0.00305604934692 s
 [GRADIENT] Time to prepare the gradient calculation: 0.0329928398132 s
 [GRADIENT] Time to call the fortran code: 0.0148189067841 s
 [GRADIENT] Time to get back in fourier space: 0.0535230636597 s

 # ---------------- NEW MINIMIZATION STEP --------------------
Step ka =  1
 [GRADIENT] Time to prepare the gradient calculation: 0.0162570476532 s
 [GRADIENT] Time to call the fortran code: 0.00587201118469 s
 [GRADIENT] Time to get back in fourier space: 0.027046918869 s
Time elapsed to symmetrize the gradient: 0.0171689987183 s
Time elapsed to compute the structure gradient: 0.000250101089478 s
Time elapsed to prepare the rho update: 0.118434906006 s
(of which to update sscha energ

Error with cmd: ssh  ekhi 'sbatch --wait /scratch/lorenzo/ESP_0.sh'
Submitted batch job 262
EXITSTATUS: 137; attempt = 1


Time elapsed to prepare the rho update: 0.113056898117 s
(of which to update sscha energies and forces: 0.0113 s)
(of which computing the Upsilon matrix: 0.0471 s)
Time elapsed to update weights the sscha energies, forces and displacements: 0.0621888637543 s
(of which to update the weights): 0.00375890731812 s
 [GRADIENT] Time to prepare the gradient calculation: 0.0287659168243 s
 [GRADIENT] Time to call the fortran code: 0.0135250091553 s
 [GRADIENT] Time to get back in fourier space: 0.0292818546295 s

 # ---------------- NEW MINIMIZATION STEP --------------------
Step ka =  148
 [GRADIENT] Time to prepare the gradient calculation: 0.0258069038391 s
 [GRADIENT] Time to call the fortran code: 0.0126891136169 s
 [GRADIENT] Time to get back in fourier space: 0.0353178977966 s
Time elapsed to symmetrize the gradient: 0.013375043869 s
Time elapsed to compute the structure gradient: 0.000355005264282 s
Time elapsed to prepare the rho update: 0.140549898148 s
(of which to update sscha ener

KeyboardInterrupt: 

The minimization is proceeding, we can study how the rombohedral angle changes as a function of the iterations, to see if it wants to go toward the symmetric phase:

In [9]:
import os
def romb_angle(dyn):
    """
    From the dynamical matrix extract the rombohedral angle in degree
    """
    v1 = dyn.structure.unit_cell[0,:]
    v2 = dyn.structure.unit_cell[2,:]
    
    cosine = v1.dot(v2) / (linalg.norm(v1) * linalg.norm(v2))
    angle = arccos(cosine)
    
    # Return the angle in degrees
    return angle * 180 / pi

# Now we can load the dynamical matrix saved along the minimization 
# and plot the value of the rombohedral angle

angles = []
for i in range(10):
    dynfile = "dyn_pop{}_".format(i)
    if os.path.exists(dynfile + "1"):
        dynmat = CC.Phonons.Phonons(dynfile)
        angles.append(romb_angle(dynmat))
        
# Now we can plot the result
plot(angles, marker = "o")
axhline(60, ls = "--", color = "k")